  
    
      
<a id='toc'></a>
<center><h1>The Battle of Neighborhood</font></h1>
Identifying Best Location for Malaysia Restaurant
Battle of Neighborhoods  
by Syazwan Faid</center>

----

## Table of Contents
- [Introduction](#introduction)
- [Objectives](#objective)
- [Data](#data)
- [Methodology](#methodology)
- [Results](#results)
- [Conclusion & Discussion](#discussion)

<a id='introduction'></a>
# Introduction
Malaysian Cuisine in among the raising interest among American. Google trends indicated that the search for Laksa and Rendang has increase in the last 5 years while the New York Fancy Food Show indicate the traction on Malay cuisine booth are increasing year by year. 


<a id='objective'></a>
# Objective
In this project, we will study in the location using Foursquare data and simple machine learning  clustering.
The aim of this project is to identify to open new restaurant by identify the nearby Malay restaurant location.

Using segmentation and clustering, we hope we can determine:
1. The density and clustering of Malaysian Restaurant in an Area.
2. The optimal location in opening Malaysian Restauran.

<a id='data'></a>
# Data
The data acquired from Foursquare Data, and will be using location of New York, Chicago, San Francisco, Jersey City, and Boston. The id for Malay Restaurant in Foursquare is 4bf58dd8d48988d156941735.

Due to limitation of the Data, the analysis is done only on the density and clustering of the data, the rental place and demand of the Malaysian Cuisine on the particular location will be done from another time.

To start, let's get and look at the data and load it to maps

Import Libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library


Log Into Foursquare

In [4]:
CLIENT_ID = 'LLS35AVA5PB5MSB3S4LAY2IDGVC1M02DEJ14ZOIWHYQ5BO0I' # your Foursquare ID
CLIENT_SECRET = '1WAL0UOHURC1ERPY05VHC0BJ3V41A0121AUJUHJKBT1JLE4T' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: LLS35AVA5PB5MSB3S4LAY2IDGVC1M02DEJ14ZOIWHYQ5BO0I


Data Scrapping

In [5]:
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'Chicago, IL', 'San Francisco, CA', 'Jersey City, NJ', 'Boston, MA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d156941735") # Malay Rest PLACE CATEGORY ID
    results[city] = requests.get(url).json()

Creating Geolocation of the Data

In [7]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

C:\Users\SyazwaneD\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


Identify Density of Malay Restaurant for Each Location

In [8]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of Malay places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of Malay places in New York, NY =  49
Showing Top 100
Total number of Malay places in Chicago, IL =  4
Showing Top 100
Total number of Malay places in San Francisco, CA =  8
Showing Top 100
Total number of Malay places in Jersey City, NJ =  54
Showing Top 100
Total number of Malay places in Boston, MA =  5
Showing Top 100


Creating Map

In [9]:
print("New York")
maps[cities[0]]

New York


In [10]:
print("Chicago")
maps[cities[1]]

Chicago


In [11]:
print("San Francisco")
maps[cities[2]]

San Francisco


Based on the data of Latitude and Longitude fo both cities, we can now create map with pointed area in it.

In [12]:
print("Jersey City")
maps[cities[3]]

Jersey City


In [13]:
print("Boston")
maps[cities[4]]

Boston


<a id='methodology'></a>
# Methodology
In this project, I will use the basic methodology as taught in Week 3 lab.   
- Above, we have done convert addresses into their equivalent latitude and longitude values.  
- Then we will use the Foursquare API to explore Malaysia Restaurant in the cities
- After that, explore function mean distance of the clustering of Malaysia Restaurant

K-means clustering algorithm will be use to complete this task. And also, the Folium library to visualize the neighborhoods in Kuala Lumpur and Johor Bahru and their emerging clusters.  


In [14]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean coordinates
0.028080995980771958
Chicago, IL
Mean Distance from Mean coordinates
0.12662295560297285
San Francisco, CA
Mean Distance from Mean coordinates
0.0982759087170497
Jersey City, NJ
Mean Distance from Mean coordinates
0.052126037579030844
Boston, MA
Mean Distance from Mean coordinates
0.06688814407594189


<a id='results'></a>
# Results                                                                         
<div style="text-align: right">[Top](#toc)</div>

In [15]:
print("New York")
maps[cities[0]]

New York


In [16]:
print("Chicago")
maps[cities[1]]

Chicago


In [17]:
print("San Francisco")
maps[cities[2]]

San Francisco


In [18]:
print("Jersey City")
maps[cities[3]]

Jersey City


In [19]:
print("Boston")
maps[cities[4]]

Boston


<a id='discussion'></a>
#  Conclusion & Discussion                                                                         
<div style="text-align: right">[Top](#toc)</div>

Based on cluster for each cities above, we believe we have found among optimal location in opening a Malaysia Restaurant in the selected cities.
The Mean Distance Value for each cities is as follows:

New York, NY Mean Distance from Mean coordinates
0.028080995980771958

Chicago, IL Mean Distance from Mean coordinates
0.12662295560297285

San Francisco, CA Mean Distance from Mean coordinates
0.0982759087170497

Jersey City, NJ Mean Distance from Mean coordinates
0.052126037579030844

Boston, MA Mean Distance from Mean coordinates
0.06688814407594189

What is lacking at this point is a systematic, quantitative way to identify and distinguish different district and to describe the correlation most optimal location encompasing the rental venue, demands and crime rate. The reality is however more complex. A further step in this classification would be to find a method to extract these common venues and integrate the spatial correlations between different of areas or district.

We believe that the classification we propose is an encouraging step towards a quantitative and systematic comparison of the different cities. Further studies are indeed needed in order to relate the data acquired, then observe it to more meaningful and objective results. 


by  
Muhamad Syazwan Faid 
Malaysia  
msyazwanfaid@gmail.com